# Classical Molecular Dynamics

Classical molecular dynamics with Crystal class.

In [5]:
import sys
sys.path.append('../') 

---

## Initialize a Crystal object

### Build the object

A crystal is built from a file containing atoms' coordinates. It must have 3 coordinates (x y z) for each row.

In [1]:
from libraries.Crystal import Crystal
# Insert a valid file name here
filename = '../../data/fcc100a108.txt'
# Initialize a Crystal object using the from_file method (constructor)
cristallo = Crystal.from_file(filename)

ModuleNotFoundError: No module named 'libraries'

### Find and store distances

With the `find_neighbours()` method it is possible to find all neighbours for each atom and their respective distances. In respect to an atom, another atom of the crystal is considered as neighbour if its distance is less than a *cutoff radious* $R_C$.

In [7]:
cristallo.find_neighbours(R_C=3) # cutoff radious = 3 Angstrom

This line of code triggers the creation of three members in the object `cristallo`:
- `cristallo.N_neighbours` 
    - list of scalars 
    - $i$-th entry is the number of neighbours of $i$-th atom
- `cristallo.which_neighbour`
    - list of lists 
    - $i$-th entry is the indexes' list of the neighbours of $i$-th atom
- `cristallo.how_distant`
    - list of lists 
    - $i$-th entry is the distances' list of $i$-th atom from his neighbours
    - the order of appearance of the distances matches the order in `.which_neighbour`; this means that its first entry .how_distant[i][0] is the distance between atom `i` and atom `.which_neighbour[i][0]`

---

## Calculate potential energy

### Theoretical background

After `find_neighbours()` execution, the neighbours and distances attributes of each atom in the crystal are populated. Having these informations, the potential energy of the system can be calculated as the sum of each couples' Lennard-Jones' potentials.
In math:
$$
V = \frac12 \sum_{i\neq j \atop i,j=1}^{N} \phi(r_{ij})
$$
where $\phi$ is Lennard-Jones' potential:
$$
\phi(r_{ij}) = 4\varepsilon\left[
    \left( \frac{\sigma}{r_{ij}} \right)^{12} -
    \left( \frac{\sigma}{r_{ij}} \right)^{6}
    \right]
$$
e.g. silver (Ag) has: $\varepsilon=0.345\ \text{eV}$ and $\sigma=2.644\ \text{\AA}$.

### Actual computation of $V$

In [8]:
# The method compute_potential() takes epsilon and sigma as arguments 
potential = cristallo.compute_potential(sigma=2.644, epsilon=0.345)
print(f"V = {potential}")

V = -154.7957616328464


---

## Calculate the forces on each atom

### Theoretical background

The force on the $k$-th atom is the partial derivative of the potential in respect to the coordinates of that atom.
$$
\vec{F}_k(\{r_{ij}\}_{ij}) = -\vec{\nabla}_k\ V(\{r_{ij}\}_{ij}) = 
-\left(
\frac{\partial V}{\partial x_k}, 
\frac{\partial V}{\partial y_k}, 
\frac{\partial V}{\partial z_k}
\right)
$$
e.g. the $x$ component of the force on the $k$-th atom is:
$$
{F_k}_x = -\frac12\frac{\partial}{\partial x_k}\sum_{i\neq j} \phi(r_{ij}) =
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k}
$$
using the *chain rule*:
$$
{F_k}_x = -\frac12\sum_{i\neq j}\frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\frac12\sum_{i\neq j}\frac{\partial\, r_{ij}}{\partial x_k}
\frac{\partial\phi(r_{ij})}{\partial\, r_{ij}}
$$
where:
$$
\begin{align*}
\frac{\partial\, r_{ij}}{\partial x_k} &= \frac{\partial}{\partial x_k}
\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{\frac12}=\\ 
&= 2(x_i-x_j)\frac{1}{2}\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{-\frac12}\delta_{ik}=\\
&= \frac{(x_i-x_j)}{r_{ij}}\delta_{ik}
\end{align*}
$$
and: 
$$
\begin{align*}
\frac{\partial\phi(r_{ij})}{\partial r_{ij}} &= \frac{\partial}{\partial r_{ij}}
4\varepsilon\left[
\left( \frac{\sigma}{r_{ij}} \right)^{12} -
\left( \frac{\sigma}{r_{ij}} \right)^{6}
\right] =\\
&=
4\varepsilon\left[
-\frac{12\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{11} +
\frac{6\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{5}
\right] =\\
&= 
4\varepsilon\left[
-12\left( \frac{\sigma^{12}}{r_{ij}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{ij}^{7}} \right)
\right]
\end{align*}
$$

In the end:
$$
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\sum_{j\neq k}\ '\ 2\varepsilon \frac{(x_k-x_j)}{r_{kj}} \left[
-12\left( \frac{\sigma^{12}}{r_{kj}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{kj}^{7}} \right)
\right]
$$
or, in a cleaner way:
$$
F_{kx} = 24\sigma^6\varepsilon\sum_{i\neq k} \frac{1}{r_{ik}^8}
\left[
\frac{2\sigma^6}{r_{ik}^6}-1
\right]
\left(
x_i-x_k
\right)
$$

### Actual computation of $\{\vec{F}_k\}_{k=0,\dots,N-1}$

In [9]:
# All this can be calculated with the compute_forces() method
vec_forze = cristallo.compute_forces(sigma=2.644, epsilon=0.345)

Now each entry of `vec_forze` is a list containing the three components (x, y, z) of the force acting on the corresponding atom. For example, `vec_forze[0]` contains the force on the first atom: `[Fx, Fy, Fz]`. As a matter of course, vector `vec_forze` has $N$ entries in total, one for each atom.